In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

## Data Pull

In [2]:
import pandas as pd

train_sheet_id = "1LummUrALksBOBaEiU30ZBWYfBfQXCQOoz2y4rk7gQms"
train_url = f"https://docs.google.com/spreadsheets/d/{train_sheet_id}/export?format=csv"

test_sheet_id = "1w6HxhTWhnvH0qfcJb4ZmzhsOiRdbHeOXWf-DYJEudUg"
test_url = f"https://docs.google.com/spreadsheets/d/{test_sheet_id}/export?format=csv"

test_df = pd.read_csv(test_url)
train_df = pd.read_csv(train_url)
#sample_submission_df = pd.read_csv("sample_submission.csv")

## EDA

In [3]:
test_df.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate
3,630003,24,male,diploma,3.03,66.3,yes,5.7,average,mixed,medium,moderate
4,630004,20,female,b.tech,2.03,42.4,yes,9.2,average,coaching,low,moderate


In [4]:
train_df.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [5]:
train_df.shape, test_df.shape

((630000, 13), (270000, 12))

In [6]:
train_df.columns


Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')

In [7]:
#null values in columns
train_df.isna().sum()

id                  0
age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
exam_score          0
dtype: int64

In [8]:
#duplicate values in the data
train_df[train_df.duplicated(keep=False)]
#no duplicate rows present

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score


In [9]:
#helps describe all the numerical columns
train_df.describe()


,id,age,study_hours,class_attendance,sleep_hours,exam_score
count,630000.000000,630000.000000,630000.000000,630000.000000,630000.000000,630000.000000
mean,314999.500000,20.545821,4.002337,71.987261,7.072758,62.506672
std,181865.479132,2.260238,2.359880,17.430098,1.744811,18.916884
min,0.000000,17.000000,0.080000,40.600000,4.100000,19.599000
25%,157499.750000,19.000000,1.970000,57.000000,5.600000,48.800000
50%,314999.500000,21.000000,4.000000,72.600000,7.100000,62.600000
75%,472499.250000,23.000000,6.050000,87.200000,8.600000,76.300000
max,629999.000000,24.000000,7.910000,99.400000,9.900000,100.000000


In [10]:
id = ['id']
cat_cols = ['gender', 'course', 'internet_access', 'sleep_quality', 'study_method', 'facility_rating', 'exam_difficulty']
num_cols = ['age', 'study_hours', 'class_attendance', 'sleep_hours']
pred_col = ['exam_score']


In [11]:
#get value_counts fro each of the categorical column
for i in cat_cols:
    print(" ")
    print(i)
    print(" ")
    print(train_df[i].value_counts())

    # data is unbalanced for categories: internet access, exam difficulty and course. 
    #data is otherwise balanced for rest of the categories. 
    
    

 
gender
 
gender
other     211097
male      210593
female    208310
Name: count, dtype: int64
 
course
 
course
b.tech     131236
b.sc       111554
b.com      110932
bca         88721
bba         75644
ba          61989
diploma     49924
Name: count, dtype: int64
 
internet_access
 
internet_access
yes    579423
no      50577
Name: count, dtype: int64
 
sleep_quality
 
sleep_quality
poor       213675
good       213089
average    203236
Name: count, dtype: int64
 
study_method
 
study_method
coaching         131697
self-study       131131
mixed            123086
group study      123009
online videos    121077
Name: count, dtype: int64
 
facility_rating
 
facility_rating
medium    214082
low       212378
high      203540
Name: count, dtype: int64
 
exam_difficulty
 
exam_difficulty
moderate    353982
easy        176540
hard         99478
Name: count, dtype: int64


In [12]:
print("Summary Statistics:")
for i in cat_cols:
    print(" ")
    print(i)
    print(train_df.groupby(i)['exam_score'].describe())


Summary Statistics:
 
gender
           count       mean        std     min   25%   50%   75%    max
gender                                                                 
female  208310.0  62.563220  18.891115  19.599  49.1  62.5  76.1  100.0
male    210593.0  62.175561  18.765553  19.599  48.5  62.5  75.8  100.0
other   211097.0  62.781191  19.087020  19.599  48.9  62.6  76.9  100.0
 
course
            count       mean        std     min   25%   50%   75%    max
course                                                                  
b.com    110932.0  62.537726  18.404936  19.599  49.4  62.2  75.9  100.0
b.sc     111554.0  62.295491  19.297236  19.599  48.3  63.1  76.1  100.0
b.tech   131236.0  62.519707  18.862381  19.599  48.7  62.4  76.6  100.0
ba        61989.0  61.886931  18.765944  19.599  47.9  62.3  75.2  100.0
bba       75644.0  63.232038  19.159462  19.599  49.4  63.6  77.6  100.0
bca       88721.0  62.586227  18.921838  19.599  48.6  62.6  76.1  100.0
diploma   49924.0 


Results as of now : 

 Unbalanced data in columns: internet access, exam difficulty and course. 

 Outliers as such there is None from IQR method. 

 now I need to find the correlation;

 first normalize the columns data and convert it into a numerical from

 I also need to check what is feature selection and how and when to do it. 


In [13]:
#for correlation matrix df.corr() gives pearon coefficient 
# Conditions for pearson and spearman coefficient - numerical data is required 
# We can also calculate te covariance - but ppearson is always the good alternative.- but it assumes linearity in the relation. 
# 1st calculate the covariance matrix 

# convert all the features to numerical values. 



In [14]:
# label_encoding on the features which has ordinal data , i.e. the rank order. 
# sleep_quality
# facility_rating
# exam_difficulty

# (rest all one hot Encoding - gender, internet_access, course, study_method)

In [15]:
#label encoding - manually for 3 categories 

# for sleep quality, facility_rating and exam_difficulty
encoded_values = {'sleep_quality' : {'poor': 0, 'good': 2, 'average': 1},
                   'facility_rating' : {'low': 0, 'medium': 1, 'high': 2},
                    'exam_difficulty': {'easy': 0, 'moderate': 1, 'hard': 2}}

In [16]:
# encode the columns based on the encoded values
for col, mapping in encoded_values.items():
    train_df[col + "_enc"] = train_df[col].map(mapping)

In [17]:
train_df.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score,sleep_quality_enc,facility_rating_enc,exam_difficulty_enc
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3,1,0,0
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7,0,1,1
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0,0,2,1
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9,1,2,1
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0,2,2,0


In [18]:
train_df.columns

Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score', 'sleep_quality_enc',
       'facility_rating_enc', 'exam_difficulty_enc'],
      dtype='object')

In [19]:
# one hot encoding for rest of the variables. 
for col in cat_cols:
    if col not in encoded_values:
        dummies = pd.get_dummies(train_df[col], prefix=col, dtype=int)
        train_df = pd.concat([train_df, dummies], axis=1)

In [20]:
train_df.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,...,course_bba,course_bca,course_diploma,internet_access_no,internet_access_yes,study_method_coaching,study_method_group study,study_method_mixed,study_method_online videos,study_method_self-study
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,...,0,0,0,1,0,0,0,0,1,0
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,...,0,0,1,0,1,0,0,0,0,1
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,...,0,0,0,0,1,1,0,0,0,0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,...,0,0,0,0,1,0,1,0,0,0
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,...,0,1,0,0,1,0,0,0,0,1


In [21]:
train_df.columns

Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score', 'sleep_quality_enc',
       'facility_rating_enc', 'exam_difficulty_enc', 'gender_female',
       'gender_male', 'gender_other', 'course_b.com', 'course_b.sc',
       'course_b.tech', 'course_ba', 'course_bba', 'course_bca',
       'course_diploma', 'internet_access_no', 'internet_access_yes',
       'study_method_coaching', 'study_method_group study',
       'study_method_mixed', 'study_method_online videos',
       'study_method_self-study'],
      dtype='object')

In [22]:
train_df.rename(columns={
    "study_method_group study": "study_method_group_study",
    "study_method_online videos": "study_method_online_videos", 
    "study_method_self-study": "study_method_self_study"
}, inplace=True)


In [23]:
#covariance matrix
num_feature_df = train_df[['id', 'age', 'study_hours', 'class_attendance',
       'sleep_hours', 'exam_score', 'sleep_quality_enc',
       'facility_rating_enc', 'exam_difficulty_enc', 'gender_female',
       'gender_male', 'gender_other', 'course_b.com', 'course_b.sc',
       'course_b.tech', 'course_ba', 'course_bba', 'course_bca',
       'course_diploma', 'internet_access_no', 'internet_access_yes',
       'study_method_coaching', 'study_method_group_study',
       'study_method_mixed', 'study_method_online_videos',
       'study_method_self_study']]
cov_matrix_pd = num_feature_df.cov()
target_cov_matrix = cov_matrix_pd[['exam_score']]

In [24]:
target_cov_matrix

,exam_score
id,1280.209869
age,0.447765
study_hours,34.028820
class_attendance,119.014916
sleep_hours,5.525598
exam_score,357.848506
sleep_quality_enc,3.688330
facility_rating_enc,2.891642
exam_difficulty_enc,0.107958
gender_female,0.018698


In [25]:
pearson_corr_matrix = num_feature_df.corr(method='pearson')
print("Pearson Correlation Matrix:")
print(pearson_corr_matrix[['exam_score']])

Pearson Correlation Matrix:
                            exam_score
id                            0.000372
age                           0.010472
study_hours                   0.762267
class_attendance              0.360954
sleep_hours                   0.167410
exam_score                    1.000000
sleep_quality_enc             0.236895
facility_rating_enc           0.188159
exam_difficulty_enc           0.008773
gender_female                 0.002101
gender_male                  -0.012403
gender_other                  0.010302
course_b.com                  0.000759
course_b.sc                  -0.005178
course_b.tech                 0.000353
course_ba                    -0.010823
course_bba                    0.014164
course_bca                    0.001703
course_diploma               -0.001587
internet_access_no           -0.000447
internet_access_yes           0.000447
study_method_coaching         0.183712
study_method_group_study     -0.051422
study_method_mixed            0.0675

In [26]:
spearman_corr_matrix = num_feature_df.corr(method='spearman')
print("Spearman Correlation Matrix:")
print(spearman_corr_matrix[['exam_score']])

Spearman Correlation Matrix:
                            exam_score
id                            0.000247
age                           0.007163
study_hours                   0.769757
class_attendance              0.351515
sleep_hours                   0.160048
exam_score                    1.000000
sleep_quality_enc             0.229405
facility_rating_enc           0.180567
exam_difficulty_enc           0.008814
gender_female                 0.000465
gender_male                  -0.010089
gender_other                  0.009619
course_b.com                 -0.001603
course_b.sc                  -0.001804
course_b.tech                -0.000354
course_ba                    -0.010453
course_bba                    0.015143
course_bca                    0.001427
course_diploma               -0.003192
internet_access_no           -0.002793
internet_access_yes           0.002793
study_method_coaching         0.175014
study_method_group_study     -0.050980
study_method_mixed            0.069

In [27]:
type(spearman_corr_matrix), type(pearson_corr_matrix), type(target_cov_matrix)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [28]:
pearson_df = (
    pearson_corr_matrix[['exam_score']]
    .reset_index()
    .rename(columns={
        'index': 'variable',
        'exam_score': 'pearson_score'
    })
)


spearman_df = (
    spearman_corr_matrix[['exam_score']]
    .reset_index()
    .rename(columns={
        'index': 'variable',
        'exam_score': 'spearman_score'
    })
)

corr_df = pd.merge(pearson_df, spearman_df, on='variable', how='inner')
print(corr_df)




                      variable  pearson_score  spearman_score
0                           id       0.000372        0.000247
1                          age       0.010472        0.007163
2                  study_hours       0.762267        0.769757
3             class_attendance       0.360954        0.351515
4                  sleep_hours       0.167410        0.160048
5                   exam_score       1.000000        1.000000
6            sleep_quality_enc       0.236895        0.229405
7          facility_rating_enc       0.188159        0.180567
8          exam_difficulty_enc       0.008773        0.008814
9                gender_female       0.002101        0.000465
10                 gender_male      -0.012403       -0.010089
11                gender_other       0.010302        0.009619
12                course_b.com       0.000759       -0.001603
13                 course_b.sc      -0.005178       -0.001804
14               course_b.tech       0.000353       -0.000354
15      

In [29]:
#threshhold >0.05 <-0.05 - this leaves us with the features of study_method, facility_rating, sleep_quality, study_hours, class_attendance, sleep_hours
corr_df_selected_features = corr_df[(corr_df['pearson_score'].abs() > 0.05) & (corr_df['spearman_score'].abs() > 0.05) & (corr_df['pearson_score'] != 1)]
print(corr_df_selected_features)

                      variable  pearson_score  spearman_score
2                  study_hours       0.762267        0.769757
3             class_attendance       0.360954        0.351515
4                  sleep_hours       0.167410        0.160048
6            sleep_quality_enc       0.236895        0.229405
7          facility_rating_enc       0.188159        0.180567
21       study_method_coaching       0.183712        0.175014
22    study_method_group_study      -0.051422       -0.050980
23          study_method_mixed       0.067598        0.069931
24  study_method_online_videos      -0.071656       -0.068261
25     study_method_self_study      -0.130269       -0.127561


In [30]:
train_df.columns

Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score', 'sleep_quality_enc',
       'facility_rating_enc', 'exam_difficulty_enc', 'gender_female',
       'gender_male', 'gender_other', 'course_b.com', 'course_b.sc',
       'course_b.tech', 'course_ba', 'course_bba', 'course_bca',
       'course_diploma', 'internet_access_no', 'internet_access_yes',
       'study_method_coaching', 'study_method_group_study',
       'study_method_mixed', 'study_method_online_videos',
       'study_method_self_study'],
      dtype='object')

In [31]:
final_df = train_df[['id', 'exam_score'] + list(corr_df_selected_features['variable']) ]

In [32]:
final_df

,id,exam_score,study_hours,class_attendance,sleep_hours,sleep_quality_enc,facility_rating_enc,study_method_coaching,study_method_group_study,study_method_mixed,study_method_online_videos,study_method_self_study
0,0,78.300,7.91,98.8,4.9,1,0,0,0,0,1,0
1,1,46.700,4.95,94.8,4.7,0,1,0,0,0,0,1
2,2,99.000,4.68,92.6,5.8,0,2,1,0,0,0,0
3,3,63.900,2.00,49.5,8.3,1,2,0,1,0,0,0
4,4,100.000,7.65,86.9,9.6,2,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
629995,629995,69.500,4.86,70.7,4.1,2,2,0,0,1,0,0
629996,629996,78.900,7.08,54.4,4.5,1,0,0,0,1,0,0
629997,629997,19.599,0.64,44.2,4.3,0,0,0,0,0,1,0
629998,629998,59.100,1.54,75.1,8.2,1,2,0,1,0,0,0


In [33]:
# final_df.to_csv(r"C:\Users\Ajay Gupta\Desktop\competitions\Student exam score\final_df.csv", index=False)

#Rough Work

In [66]:
# import matplotlib.pyplot as plt

# def plot_category_distribution(
#     df,
#     category_col,
#     numeric_col,
#     bins=80,
#     min_kde_samples=20
# ):
#     categories = df[category_col].dropna().unique()

#     # =========================
#     # 1️⃣ SIDE-BY-SIDE PLOTS
#     # =========================
#     fig, axes = plt.subplots(
#         1,
#         len(categories),
#         figsize=(6 * len(categories), 5),
#         sharey=True
#     )

#     if len(categories) == 1:
#         axes = [axes]

#     for ax, cat in zip(axes, categories):
#         data = df[df[category_col] == cat][numeric_col].dropna()

#         ax.hist(
#             data,
#             bins=bins,
#             density=True,
#             alpha=0.6
#         )

#         if len(data) >= min_kde_samples:
#             data.plot(kind='kde', ax=ax)

#         ax.set_title(f"{category_col} = {cat}")
#         ax.set_xlabel(numeric_col)
#         ax.set_ylabel("Density")

#     plt.tight_layout()
#     plt.show()

#     # =========================
#     # 2️⃣ OVERLAPPING PLOT
#     # =========================
#     plt.figure(figsize=(10, 6))

#     for cat in categories:
#         data = df[df[category_col] == cat][numeric_col].dropna()

#         plt.hist(
#             data,
#             bins=bins,
#             density=True,
#             alpha=0.35,
#             label=str(cat)
#         )

#         if len(data) >= min_kde_samples:
#             data.plot(kind='kde', linewidth=2)

#         # Mean line
#         plt.axvline(
#             data.mean(),
#             linestyle='--',
#             alpha=0.7
#         )

#     plt.title(f"Overlapping {numeric_col} Distribution by {category_col}")
#     plt.xlabel(numeric_col)
#     plt.ylabel("Density")
#     plt.legend()
#     plt.show()

#     # =========================
#     # 3️⃣ SUMMARY STATS
#     # =========================
#     print("Summary Statistics:")
#     display(
#         df.groupby(category_col)[numeric_col].describe()
#     )


In [67]:
# plot_category_distribution(
#     train_df,
#     category_col='internet_access',
#     numeric_col='exam_score'
# )


In [68]:
# plot_category_distribution(
#     train_df,
#     category_col='gender',
#     numeric_col='exam_score'
# )


In [69]:
# plot_category_distribution(
#     train_df,
#     category_col='course',
#     numeric_col='exam_score'
# )


In [70]:


# plt.figure(figsize=(8, 6))
# plt.scatter(
#     train_df['study_hours'],
#     train_df['exam_score'],
#     alpha=0.5
# )

# plt.title('Study Hours vs Exam Score')
# plt.xlabel('Study Hours')
# plt.ylabel('Exam Score')
# plt.show()


In [71]:
# sorted_df = train_df.sort_values('study_hours')

# plt.figure(figsize=(8, 6))
# plt.plot(
#     sorted_df['exam_score'],
#     sorted_df['study_hours'],
#     alpha=0.6
# )

# plt.title('Study Hours vs Exam Score (Line Plot)')
# plt.xlabel('Study Hours')
# plt.ylabel('Exam Score')
# plt.show()
